### NYC NY taxi data EDA

- Q1:Call numbers per hours?
- Q2:Call numbers per weekdays?
- Use GCP Bigquery, data visualization (seaborn, plotly)

In [3]:
# !pip3 install chart-studio cufflinks pandas numpy seaborn matplotlib pandas-gbq

     |████████████████████████████████| 64 kB 496 kB/s eta 0:00:01


You should consider upgrading via the '/usr/local/bin/python3.7 -m pip install --upgrade pip' command.


In [1]:
import chart_studio.plotly as py
import cufflinks as cf
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

plt.style.use('ggplot')
print(cf.__version__)

%config InlineBackend.figure_format = 'retina'
cf.go_offline()

0.17.3


### 1. Whole trip numbers in January

In [2]:
import os
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '~/.config/gcloud/application_default_credentials.json'

In [3]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/workspace/AI_Data_Science/mobility-285808-aa8e44818175.json'

In [5]:
# import GCP Client library
from google.cloud import bigquery

In [7]:
#instantiate a client for bigquery service
bigquery_client = bigquery.Client()
bigquery_client

In [8]:
# Query for Bigquery
query = """
SELECT 
    DATETIME_TRUNC(pickup_datetime, hour) as pickup_hour,
    count(*) as cnt
FROM `bigquery-public-data.new_york_taxi_trips.tlc_yellow_trips_2018`
WHERE EXTRACT(MONTH from pickup_datetime) = 1
GROUP BY pickup_hour
ORDER BY pickup_hour
"""

In [9]:
# run query and get data from bigquery
query_job = bigquery_client.query(query) #API request

In [10]:
# data into dataframe
df = query_job.to_dataframe()
df.head()

,pickup_hour,cnt
0,2001-01-01 00:00:00,10
1,2001-01-01 22:00:00,2
2,2001-01-05 11:00:00,1
3,2001-01-06 06:00:00,1
4,2003-01-01 00:00:00,13
